<a href="https://colab.research.google.com/github/e11106013/LLM/blob/main/Testing_royal_ZhTW_ID_q2_k_GGUFv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 安裝軟體套件
!pip install llama-cpp-python gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127094 sha256=f1ae75f652c0ff51c34e345a1b47b532e59a3d2c6ac24c8487a1bfe7068af94c
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [2]:
#@title 登入HF
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [3]:
#@title 下載自HF下載模型
!mkdir -p /content/models
#!huggingface-cli download roylin1003/royal-ZhTW-ID-q4_k_m --local-dir models
!huggingface-cli download roylin1003/royal-zhTW-ID-q2_k.gguf --local-dir models

Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading '.gitattributes' to 'models/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.b5c77c434637aeb093f8efb52f80580613bb5248.incomplete'

.gitattributes: 100% 1.58k/1.58k [00:00<00:00, 13.2MB/s]
Download complete. Moving file to models/.gitattributes
Fetching 3 files:  33% 1/3 [00:00<00:00,  3.16it/s]
royal-zhTW-ID-q2_k.gguf:   0% 0.00/3.02G [00:00<?, ?B/s]

README.md: 100% 3.00k/3.00k [00:00<00:00, 21.9MB/s]
Download complete. Moving file to models/README.md
Fetching 3 files:  67% 2/3 [00:00<00:00,  5.03it/s]
royal-zhTW-ID-q2_k.gguf:   0% 10.5M/3.02G [00:00<01:21, 37.0MB/s]
royal-zhTW-ID-q2_k.gguf:   1% 21.0M/3.02G [00:00<01:03, 46.9MB/s]
royal-zhTW-ID-q2_k.gguf:   1% 31.5M/3.02G [00:00<00:58, 51.2MB/s]
royal-zhTW-ID-q2_k.gguf:   1% 41.9M/3.02G [00:00<00:54, 54.1MB/s]
royal-zhTW-ID-q2_k.gguf:   2% 52.4M/3.02G [00:01<00:53, 55.2MB/s]
royal-zhTW-ID-q2_k.gguf:   2% 62.9M/3.02G [00:01<00:52, 55.8MB/s]
royal-zhTW-ID-q2_k.gguf:   2% 73

In [7]:
import gradio as gr
from llama_cpp import Llama

# --- 載入模型（使用 cache 避免重複載入） ---
def load_model():
    return Llama(
        model_path="models/royal-zhTW-ID-q2_k.gguf",
      #  n_gpu_layers=20,
        n_gpu_layers=-1,    # 使用所有 GPU 層（-1 表示全部）

        n_ctx=2048
    )

llm = load_model()

# --- 翻譯函式 ---
def translate(input_text, direction, history):
    if direction == "華語 ➜ 印尼語":
        prompt = f"""請將下列華語句子精準翻譯為印尼語，只回傳印尼語內容，不要翻譯成其他語言：
華語：{input_text.strip()}
印尼語："""
        output_prefix = "印尼語："
    else:
        prompt = f"""請將下列印尼語句子精準翻譯為華語，只回傳華語內容，不要翻譯成其他語言：
印尼語：{input_text.strip()}
華語："""
        output_prefix = "華語："

    # 模型推論
    output = llm(prompt, max_tokens=100)
    result = output["choices"][0]["text"].strip()

    user_display = f"👤 {input_text.strip()}"
    bot_display = f"🤖 {output_prefix}{result}"

    history.append((user_display, bot_display))
    return "", history

# --- Gradio 介面設定 ---
with gr.Blocks(title="繁中印尼翻譯助理") as demo:
    gr.Markdown("# 🗣️ 繁中 ↔ 印尼 翻譯對話介面")
    gr.Markdown("使用模型：`roylin1003/royal-ZhTW-ID-q4_k_m`")

    direction = gr.Radio(
        ["華語 ➜ 印尼語", "印尼語 ➜ 華語"],
        label="請選擇翻譯方向",
        value="華語 ➜ 印尼語"
    )

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入要翻譯的句子...", label="輸入")
    state = gr.State([])

    msg.submit(translate, [msg, direction, state], [msg, chatbot])

# --- 執行 ---
demo.launch()

llama_model_loader: loaded meta data with 27 key-value pairs and 339 tensors from models/royal-zhTW-ID-q2_k.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Qwen
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   6:                         general.size_label str              = 7B
llama_model_loader: - kv   7:                          qwen2.

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7489376078f28fc982.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
